<a href="https://colab.research.google.com/github/MariaOmaechevarria/OPENCL/blob/main/FILTROS%20IMAGENES/Filtro_median.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt purge *nvidia* -y
!sudo apt install nvidia-driver-530 -y


!pip install pyopencl
!apt-get install -y pocl-opencl-icd ocl-icd-libopencl1

In [ ]:
import pyopencl as cl
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

from PIL import Image, ImageFilter

In [ ]:
kernel_median="""
__kernel void kernel_median(__global uchar* imagen_in, __global uchar* imagen_out, int dim, int ancho, int alto) {

    int fila = get_global_id(0);
    int columna = get_global_id(1);

    int centro = (dim - 1) / 2;
    int i, j;

    // Asegurarse de que el píxel esté dentro de los límites
    if (centro <= fila && fila < (alto - centro) && centro <= columna && columna < (ancho - centro)) {

        // Variables privadas (para cada hilo) para almacenar los píxeles vecinos

        uchar ventana_rojo[25]; // Asumiendo una ventana máxima de 5x5,depende de dim
        uchar ventana_verde[25];
        uchar ventana_azul[25];

        //Numero total de elementos en la ventana
        int count = 0;

        // Recorrer la ventana de tamaño `dim x dim`
        for (i = -centro; i <= centro; i++) {
            for (j = -centro; j <= centro; j++) {

                // Calcular el índice del píxel en la imagen de entrada
                int idx = ((fila + i) * ancho + (columna + j)) * 3;

                // Guardar los valores de los píxeles en las ventanas

                ventana_rojo[count] = imagen_in[idx];
                ventana_verde[count] = imagen_in[idx + 1];
                ventana_azul[count] = imagen_in[idx + 2];
                count++;
            }
        }

        // Ordenar los valores de los píxeles en cada canal

        for (int k = 0; k < count - 1; k++) {
            for (int l = k + 1; l < count; l++) {

                if (ventana_rojo[k] > ventana_rojo[l]) {
                    uchar temp = ventana_rojo[k];
                    ventana_rojo[k] = ventana_rojo[l];
                    ventana_rojo[l] = temp;
                }
                if (ventana_verde[k] > ventana_verde[l]) {
                    uchar temp = ventana_verde[k];
                    ventana_verde[k] = ventana_verde[l];
                    ventana_verde[l] = temp;
                }
                if (ventana_azul[k] > ventana_azul[l]) {
                    uchar temp = ventana_azul[k];
                    ventana_azul[k] = ventana_azul[l];
                    ventana_azul[l] = temp;
                }
            }
        }

        // Asignar el valor central al píxel de salida

        int idx_out = (fila * ancho + columna) * 3;
        imagen_out[idx_out] = ventana_rojo[count / 2];
        imagen_out[idx_out + 1] = ventana_verde[count / 2];
        imagen_out[idx_out + 2] = ventana_azul[count / 2];

    } else {
        // Si el píxel está fuera de los límites, mantener el valor original

        int idx_out = (fila * ancho + columna) * 3;
        imagen_out[idx_out] = imagen_in[idx_out];
        imagen_out[idx_out + 1] = imagen_in[idx_out + 1];
        imagen_out[idx_out + 2] = imagen_in[idx_out + 2];
    }
}

"""

In [ ]:
def importar_imagen(path):
  imagen = Image.open(path)
  return imagen

In [ ]:
def aplicar_filtro_color_median(image_path,  kernel_code, kernel_name, device_type, local_size):
    # Importar imagen
    imagen = importar_imagen(image_path)

    # Convertirla a un array de tres canales
    imagen_np = np.array(imagen).astype(np.uint8)

    # Dimensiones de la imagen
    tam_x, tam_y, _ = imagen_np.shape

    # Crear array para la imagen final
    imagen_out_np = np.empty_like(imagen_np)

    # Plataforma y dispositivo
    platform = cl.get_platforms()[0]
    device = platform.get_devices(device_type=device_type)[0]

    # Crear contexto y cola de comandos
    context = cl.Context([device])
    command_queue = cl.CommandQueue(context, device=device, properties=cl.command_queue_properties.PROFILING_ENABLE)

    # Crear el programa y compilarlo
    program = cl.Program(context, kernel_code).build()

    # Crear el kernel
    kernel_filter = cl.Kernel(program, kernel_name)

    # Crear buffers de imagen
    buffer_in = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=imagen_np)
    buffer_out = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, imagen_out_np.nbytes)


    # Argumentos del kernel
    kernel_filter.set_arg(0, buffer_in)
    kernel_filter.set_arg(1, buffer_out)
    kernel_filter.set_arg(2, np.int32(3))
    kernel_filter.set_arg(3, np.int32(imagen_np.shape[1]))
    kernel_filter.set_arg(4, np.int32(imagen_np.shape[0]))

    # Ejecutar el kernel
    global_size = (tam_x, tam_y)
    event = cl.enqueue_nd_range_kernel(command_queue, kernel_filter, global_size, local_size)
    event.wait()

    # Leer el buffer de salida
    cl.enqueue_copy(command_queue, imagen_out_np, buffer_out)

    # Obtener el tiempo de ejecución
    exec_time = 1e-9 * (event.profile.end - event.profile.start)

    # Guardar y mostrar la imagen resultante
    imagen_resultante = Image.fromarray(imagen_out_np)
    imagen_resultante.show()
    #imagen_resultante.save("/content/drive/My Drive/Colab Notebooks/TFG_OPENCL/FILTROS IMAGENES/pirineos_filtro_median.png")

    return imagen_resultante, exec_time


In [ ]:

def filtro_median():
    device_type = cl.device_type.CPU
    image_path = "/content/drive/My Drive/Colab Notebooks/TFG_OPENCL/FILTROS IMAGENES/IMAGENES/pirineos_imagen.png"

    # Definir las combinaciones de local_size, incluyendo (1,1), (2,2), (4,4), (8,8), (16,16)
    local_sizes = [(1, 1), (2, 2), (4, 4), (8, 8), (16, 16), (16, 8), (8, 16)]

    # Crear columnas para el DataFrame
    columns = [f"({ls[0]}/{ls[1]})" for ls in local_sizes]
    results_df = pd.DataFrame(index=['Imagen Pirineos'], columns=columns)

    # Probar cada local_size y medir el tiempo de ejecución
    for local_size in local_sizes:
        imagen_resultante, exec_time = aplicar_filtro_color_median(image_path, kernel_median, "kernel_median", device_type, local_size)

        # Asignar el tiempo de ejecución al DataFrame
        results_df.loc['Imagen Pirineos', f"({local_size[0]}/{local_size[1]})"] = exec_time if exec_time is not None else "NP"
        print(f"local_size: {local_size}, tiempo de ejecución: {exec_time}")

    # Guardar los resultados en un archivo CSV
    results_df.to_csv('/content/drive/My Drive/Colab Notebooks/TFG_OPENCL/FILTROS IMAGENES/Filtros_median_CPU.csv', index=True)

filtro_median()


local_size: (1, 1), tiempo de ejecución: 0.22100033400000002
local_size: (2, 2), tiempo de ejecución: 0.237080702
local_size: (4, 4), tiempo de ejecución: 0.22843216000000002
local_size: (8, 8), tiempo de ejecución: 0.24456481600000002
local_size: (16, 16), tiempo de ejecución: 0.245580013
local_size: (16, 8), tiempo de ejecución: 0.242636536
local_size: (8, 16), tiempo de ejecución: 0.24519632900000002
